In [4]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [5]:
cooc = pd.read_csv("matrice_co-occurence.csv", index_col="Unnamed: 0")


#fichier csv qui sera merge pas les prédiction et soumis sur Kaggle

In [6]:
po = pd.read_csv("presences_only_train.csv", sep=';')
blind = pd.read_csv("test_blind.csv", sep=";")

In [7]:
spe = po["speciesId"].unique()
liste_pa = cooc.index

in
['9201', '1991', '2181', '3145', '9330', '55', '9474', '7558', '3904', '4166', '5200', '5989', '307', '2183', '6273', '9118', '5361', '2950', '5314', '970', '7571', '9019', '8001', '8161', '4091', '4313', '9588', '9655', '8417', '5296', '7249', '7560', '4525', '7057', '3069', '7854', '4057', '4284', '8761', '9248', '2875', '9124', '747', '2959', '6155', '2625', '9514', '1849', '9071', '6107']
[0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 0.0004599816007359, 

In [8]:
data_sans_doublons = po.groupby(['lat', 'lon'])['speciesId'].apply(list).reset_index()

In [9]:
def k_voisins(k, data_sans_doublons, test):
    """
    données:
        k: nombre de voisins considérés 
        data: dataFrame contenant entre autre les patch_ID et les espèces associées du train-PA
        test: dataFrame provenant du CSV Kaggle contenant les informations sur les lieux où la prédiction
        doit être faite
    fonction: 
        Liste les espèces présentes dans les k-voisins les plus proches de chaque coordonnées du dataFrame
        test à partir des information de data. 
    résultat:
        resultat: Un fichier CSV contenant les prédicitions d'especes dans la colonne Predicted 
    """
    # data frame contenant les espèces_id selon la lat,lon où l'espèce est recensées (sans doublons)
    #data_sans_doublons = data.groupby(['lat', 'lon'])['speciesId'].apply(list).reset_index()
    # Données utilisées par l'algo 
    X1 = test[['lat', 'lon']].values
    X2 = data_sans_doublons[['lat', 'lon']].values
    # Création du modèle
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X2)
    # le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
    distances, indices = nn.kneighbors(X1)
    
    # Récupération des espèces selon les indices des coordonnées des voisins
    result = []
    for i in range(len(test)):
        nearest_species = {}
        for j in range(len(indices[i])):
            index = indices[i][j]
            #nearest_species.update(data_sans_doublons.loc[index]['speciesId'])
            for l in range (0,len(data_sans_doublons.loc[index]['speciesId'])):
                if data_sans_doublons.loc[index]['speciesId'][l] in nearest_species :
                    nearest_species[data_sans_doublons.loc[index]['speciesId'][l]] += 1
                else :
                    nearest_species[data_sans_doublons.loc[index]['speciesId'][l]] = 1
        result.append(nearest_species)
    
    #traitement pour l'export du CSV au format acceptable par Kaggle
    test['nearest_species'] = result
   # test = test.rename(columns={'nearest_species': 'Predicted'})    

    return test


In [65]:
k = 200

In [66]:
%%time
df = k_voisins(k, data_sans_doublons, blind)
#df.head()

CPU times: total: 18min 13s
Wall time: 18min 20s


,Id,datasetName,date,dayOfYear,year,patchID,timeSerieID,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,nearest_species,Prediction,Predicted
0,1,CBNMed,1629064800,228,2021,10125,3988380,1.729607,42.52171,3640762.0,2194746.0,5.0,"{9202: 1, 7925: 1, 528: 1, 2959: 1, 7558: 1, 1...","8146,7262,3956","9202, 7925, 528, 2959, 7558, 1376, 5169, 1629,..."
1,2,CBNMed,1629064800,228,2021,10126,3997090,1.729624,42.52177,3640764.0,2194753.0,4.0,"{9202: 1, 7925: 1, 528: 1, 2959: 1, 1629: 3, 7...","8146,7262,3956","9202, 7925, 528, 2959, 1629, 7558, 1376, 5169,..."
2,3,CBNMed,1629064800,228,2021,10127,4001106,1.729652,42.52197,3640769.0,2194775.0,5.0,"{528: 1, 9202: 1, 7925: 1, 1629: 3, 2959: 1, 1...","8146,7262,3956","528, 9202, 7925, 1629, 2959, 1966, 7558, 1376,..."
3,4,CBNMed,1629064800,228,2021,10129,4004605,1.729671,42.52192,3640770.0,2194769.0,6.0,"{528: 1, 9202: 1, 7925: 1, 1629: 3, 2959: 1, 7...","8146,7262,3956","528, 9202, 7925, 1629, 2959, 7558, 1376, 5169,..."
4,5,CBNMed,1531346400,193,2018,10154,3987660,1.826847,42.57452,3649316.0,2199695.0,0.0,"{4204: 1, 9920: 1, 9372: 2, 8106: 1, 7262: 9, ...","7262,7319,3956,133,5352","4204, 9920, 9372, 8106, 7262, 7319, 9193, 3956..."


In [67]:
%%time
def creation_dict2(df, comat) :
    liste_dict = []
   
    for i in range(0, len(df)) :
        big_dict = {}
        for key in df.loc[i].nearest_species :
            if key not in big_dict :
                big_dict[key]={}
        for key in df.loc[i].nearest_species :
            if key in liste_pa :
                especes = cooc.loc[key][cooc.loc[key] > 0.0].index.to_list()
                proba = cooc.loc[key][cooc.loc[key] > 0.0].to_list()
                for i in range(0,len(especes)) :
                    big_dict[key][especes[i]] = proba[i]
        liste_dict.append(big_dict)
    return liste_dict

dicts2 = creation_dict2(df, cooc)

CPU times: total: 19min 53s
Wall time: 22min 21s


In [68]:
for i in range(0, len(dicts1)) :
    for key in dicts1[i] :
        dicts1[i][key] = dicts1[i][key]/k


In [73]:
%%time
# Dicts1 et dicts2 de même longueur, indice i correspond à même ligne du data frame

colonne_predicted = []
for i in range(0, len(dicts1)) :
    predicted = [] 
    not_added = {}
    for key in dicts1[i] :
        if key not in predicted and dicts1[i][key]>0.1 :
            predicted.append(key)
        elif key not in predicted :
            not_added[key] = dicts1[i][key]
  #  for key in dicts2[i] :
  #      for key2 in dicts2[i][key] :
  #          if key2 not in predicted :
                
    row_dict = {}
    for key in dicts2[i] :
        if key not in row_dict :
            row_dict[key] = dicts2[i][key]
        else :
            row_dict[key] += dicts2[i][key]
    for key in row_dict :
        for subkey in row_dict[key] :
            if row_dict[key][subkey] > 0.1 and subkey not in predicted:
                predicted.append(subkey)
            elif subkey in not_added and row_dict[key][subkey]+not_added[subkey] > 0.1 and subkey not in predicted :
                predicted.append(subkey)
    colonne_predicted.append(predicted)

#print(colonne_predicted)

CPU times: total: 46.5 s
Wall time: 10min 11s


In [74]:
nvl_list = []
for el in colonne_predicted :
    str_pred = ""
    for i in range(0, len(el)) :
        if i!=len(el)-1 :
            str_pred+= str(el[i])+", "
        else :
            str_pred+= str(el[i])
    nvl_list.append(str_pred)

nvl_list

['5959, 8935, 5763, 6361',
 '5959, 8935, 5763, 6361',
 '5959, 8935, 5763, 6361',
 '5959, 8935, 5763, 6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '6361',
 '5959, 8935, 6361, 4284, 2509, 3069, 6574, 795, 4064, 9330, 8079, 7648, 5989, 9347, 386, 9118, 9329, 1005, 1283, 5263, 9906, 1333',
 '8935, 4284, 6361, 5959, 5763',
 '8935, 4284, 6361, 5959, 5763',
 '6361, 5959, 8935, 4284, 2509, 3069, 6574, 795, 4064, 9330, 8079, 7648, 5989, 9347, 386, 9118, 9329, 1005, 1283, 5263, 9906, 1333',
 '5959, 8935, 4284, 6574, 795, 4064, 9330, 8079, 7648, 5989, 9347, 386, 9118, 9329, 1005, 2509, 1283, 5263, 3069, 9906, 1333, 6361, 4906, 24, 5763',
 '5959, 8935, 4284, 6361, 6574, 795, 4064, 9330, 8079, 7648, 5989, 9347, 386, 9118, 9329, 1005, 2509, 1283, 5263, 3069, 9906, 1333, 4906, 24, 5763',
 '5959, 8935, 6574, 795, 4064, 9330, 8079, 7648, 5989, 9347, 386, 9118, 9329, 1005, 2509, 1283, 5263, 3069, 9906, 1333, 4284, 4906, 24, 5763, 6361',
 '795, 4064, 8079, 7648

In [75]:
df["Predicted"] = nvl_list

In [76]:
submit = df[["Id", "Predicted"]]
submit = submit.groupby(by='Id').sum()
submit


,Predicted
Id,
1,"5959, 8935, 5763, 6361"
2,"5959, 8935, 5763, 6361"
3,"5959, 8935, 5763, 6361"
4,"5959, 8935, 5763, 6361"
5,6361
...,...
22400,"5959, 8935, 4218, 386, 5314, 1005, 2509, 4284,..."
22401,"5959, 8935, 6361"
22402,


In [77]:
submit.to_csv("modele_po_co-occ.csv")

In [7]:
df2 = df.copy()
df2 = df2[['Id', 'Predicted']]

predictions = df.Predicted.tolist()

In [8]:
def recup_cooc(ID, mat) :
    indices_lignes = cooc.loc[cooc[str(ID)] > 0].index
    print(indices_lignes)
    valeurs = cooc.loc[cooc[str(ID)] > 0].values
    print(valeurs)
    result = [indices_lignes, valeurs]
    return result

recup_cooc(6574, cooc)

Int64Index([1007,  795, 4715, 9286,  876, 4064,  904, 5325, 8481, 7512,
            ...
             536, 4397, 4961, 3986,  494,  977, 2263, 6107, 1232, 8547],
           dtype='int64', length=534)
[[0.00137994 0.         0.         ... 0.         0.         0.        ]
 [0.02345906 0.00045998 0.         ... 0.         0.         0.        ]
 [0.00045998 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.02621895 0.         0.         ... 0.         0.         0.        ]
 [0.01471941 0.         0.         ... 0.         0.         0.        ]
 [0.00689972 0.         0.         ... 0.         0.         0.        ]]


[Int64Index([1007,  795, 4715, 9286,  876, 4064,  904, 5325, 8481, 7512,
             ...
              536, 4397, 4961, 3986,  494,  977, 2263, 6107, 1232, 8547],
            dtype='int64', length=534),
 array([[0.00137994, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.02345906, 0.00045998, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00045998, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.02621895, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.01471941, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00689972, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])]

In [9]:
for predict in predictions :
    # On
    for especes in predict :
        

SyntaxError: incomplete input (601272456.py, line 4)

In [ ]:
for espece in df.Predicted :
    for esp in espece :
        # Pour chaque espece de la liste de predicted 
        for cooccur in cooc.iloc[esp] :
            if cooccur > 0 & cooc.index != esp:
                df2.Predicted[df2.Predicted==esp].append(cooc.index)
df2.copy